# UVV/DBZ Threshold Occurence Map

In [1]:
import pandas as pd
import xarray as xr
import glob
import numpy as np
from copy import deepcopy



In [2]:
df = pd.concat(map(pd.read_csv, glob.glob(f'/home/scratch/jcorner1/Thesis/future_modes/modes/HIST/*')), ignore_index=True)
df['Time'] = pd.to_datetime(df['Time'])

In [3]:
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
uhs = []

#iterate through the dataframes
for did, day in df.resample('24h', origin='1990-10-01 12:00:00', on='Time'):

    results = np.zeros(shape=(899, 1399))
    results[day.y.values.astype(int), day.x.values.astype(int)] = 1
    uh_uvv = deepcopy(geog)
    uh_uvv = uh_uvv.assign_coords({'Time': np.array([did])})
    uh_uvv = uh_uvv.assign(UVV_DAYS = (('Time', 'south_north', 'west_east'), np.expand_dims(results, axis=0)))
    uh_uvv = uh_uvv[['CLAT', 'CLONG', 'UVV_DAYS']].coarsen(south_north=20, west_east=20, boundary='trim').mean()
    uh_uvv['UVV_DAYS'] = 1*(uh_uvv['UVV_DAYS'] > 0)

    uhs.append(uh_uvv)

ds = xr.concat(uhs, dim='Time')
